In [1]:
print("hello")

hello


In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
groq_api_key = os.environ['GROQ_API_KEY']
hf_token_api = os.environ['HF_TOKEN']
pinecone_api_key = os.environ['PINECONE_API_KEY']

In [3]:
from langchain_groq import ChatGroq
Model = ChatGroq(
    model="llama3-70b-8192"
)

#### RunnablePassthrough
- it does not do anything to the input data.
-a simple exemple chain with RunnablePassthrough() will output the original input without any modification 

In [5]:
from langchain_core.runnables import RunnablePassthrough

chain = RunnablePassthrough()

In [6]:
chain.invoke('Dan')

'Dan'

### RunnableLambda
- To use a custom function inside a LCEL chain we need to warp it up with RunnableLambda
- let's make a simple function exemple create a filipino lastnames

In [8]:
def filipino_lastname(name : str) -> str:
    return f"Son {name}"

In [9]:
filipino_lastname("Suchesa")

'Son Suchesa'

In [10]:
from langchain_core.runnables import RunnableLambda
chain = RunnablePassthrough() | RunnableLambda(filipino_lastname)

In [11]:
chain.invoke("Suuchesa")

'Son Suuchesa'

### RunnableParallel

- We will use `RunnableParallel()` for running tasks in parallel.
- This is probably the most important and most useful `Runnable` from LangChain.
- In the following chain, `RunnableParallel` is going to run these two tasks in parallel:
  - `operation_a` will use `RunnablePassthrough`.
  - `operation_b` will use `RunnableLambda` with the `filipino_lastname` function.


In [12]:
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel({
    'operation_a': RunnablePassthrough(),
    'operation_b': RunnableLambda(filipino_lastname)
})

In [13]:
chain.invoke("Dan")

{'operation_a': 'Dan', 'operation_b': 'Son Dan'}

In [14]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [15]:
template = """
Tell me a curious fact about {soccer_player}
"""

prompt = ChatPromptTemplate.from_template(template=template)

output_parser = StrOutputParser()

In [16]:
def filipino_lastname_from_dict(person):
    return person['name'] + 'Son'


In [17]:
chain = RunnableParallel({
    'operation_a': RunnablePassthrough(),
    'soccer_player': RunnableLambda(filipino_lastname_from_dict),
    'operation_c' : RunnablePassthrough(),
    
}) | prompt | Model | output_parser

In [18]:
chain.invoke({
    'name1' : 'Jordam',
    "name": "Nina"
})

'I apologize, but I\'m not familiar with a person or entity called "NinaSon." Could you be referring to Nina Simone, the legendary American singer, songwriter, and civil rights activist? If so, here\'s a curious fact about her:\n\nNina Simone was a classically trained pianist and was accepted into the Juilliard School of Music in New York City, but she was denied a scholarship to study at the Curtis Institute of Music in Philadelphia due to her race. This rejection was a pivotal moment in her life, leading her to focus on jazz and popular music instead.'

### Advance use case of RunnableParallel

In [23]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough ,RunnableParallel
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_groq import ChatGroq

In [24]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_embeddings = HuggingFaceEmbeddings(model_name=model_name)

In [ ]:
vector_store = FAISS.from_texts([
    'Dan focuses on providing content on Data Science , AI, ML, DL, NLP, and python programming , ect, in English'], embedding = model_embeddings)

retriever = vector_store.as_retriever()

template = """
Answer the question based only on the following context:
{context}

Question : {question}
"""

prompt = ChatPromptTemplate.from_template(template=template)


Model = ChatGroq(
    model="llama3-70b-8192"
)

retrieval_chain = (
    RunnableParallel({'context': retriever, 'question': RunnablePassthrough()})
    |prompt
    |Model
    |StrOutputParser()
    
) 


In [26]:
print(retrieval_chain.invoke('what is Dan'))

Based on the provided context, it can be inferred that Dan is a content provider, specifically focusing on providing content related to Data Science, AI, ML, DL, NLP, and Python programming.


### Let's use itemgetter from operator

In [28]:
from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough ,RunnableParallel
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_groq import ChatGroq

In [29]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_embeddings = HuggingFaceEmbeddings(model_name=model_name)

vector_store = FAISS.from_texts([
    'Dan focuses on providing content on Data Science , AI, ML, DL, NLP, and python programming , ect, in English'], embedding = model_embeddings)

retriever = vector_store.as_retriever()

template = """
Answer the question based only on the following context:
{context}

Question : {question}

Answer in the following language : {language}
"""

prompt = ChatPromptTemplate.from_template(template=template)


Model = ChatGroq(
    model="llama3-70b-8192"
)

retrieval_chain = (
    {
        'context': itemgetter('question') | retriever,
        'question': itemgetter('question'),
        'language':  itemgetter('language'), 
    }
    |prompt
    |Model
    |StrOutputParser()
    
) 

In [30]:
retrieval_chain.invoke({'question': 'what is Dan', 'language': 'hebrew'})

'דן הוא מומחה בתחום המדעי הנתונים והתכנות.'